In [1]:
from sodapy import Socrata
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions,StandardOptions
from datetime import datetime,timedelta
import os
import json

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/prabha/GitHub/chicago_crime/beam/crime_batch_key.json"

In [3]:
table_schema= 'id:numeric,case_number:string,date:datetime,block:string,iucr:string,primary_type:string,description:string,location_description:string,arrest:boolean,domestic:boolean,beat:string,district:string,ward:string,community_area:string,fbi_code:string,x_coordinate:numeric,y_coordinate:numeric,year:numeric,updated_on:datetime,latitude:float,longitude:float,location:string'

In [4]:
class fetch_data(beam.DoFn):
    def process(self,url):
        
        updated_last_n_days = 1
        client = Socrata(url, app_token="Ttz4HIh52J3g53HKTYKMNxu4M")

        # Get all the updates in the last week.
        updated_on_filter = "updated_on >= '"+datetime.strftime(datetime.today().date()-timedelta(days = updated_last_n_days),'%Y-%m-%d')+"T00:00:00.000'"

        crimes = list(client.get_all("ijzp-q8t2",content_type='json',where = updated_on_filter,))
        
        crimes = crimes[1:] # remove the header row
        
        return crimes
    

In [5]:
def get_first_element(row):
    return row.split(',')

In [6]:
def data_type_conversion(row):
    int_type = ['id','x_coordinate','y_coordinate']
    float_type = ['latitude','longitude']
    
    for col in int_type:
        if col in row:
            row[col] = int(row[col])
        else:
            row[col] = 0
            
    for col in float_type:
        if col in row:
            row[col] = float(row[col])
        else:
            row[col] = 0.0
    
    if 'location' in row:
        row['location'] = json.dumps(row['location'])
    else:
        row['location']=""

    return row

In [7]:

p1 = beam.Pipeline()

cleaned_data = (
    p1
    | beam.Create(['data.cityofchicago.org'])
    | "Call API" >> beam.ParDo(fetch_data())
   # | "Get first element" >> beam.FlatMap(get_first_element)
)


#table_schema='id:string'

write_to_bq = (
    cleaned_data
    | "Clean Data to Json" >> beam.Map(data_type_conversion)
    #| beam.io.WriteToText("output")
    | beam.io.WriteToBigQuery('chicago-crime-284514:data_lake.chicago_crime',schema=table_schema,
                              create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
                              write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
                              custom_gcs_temp_location='gs://chicago-crime-284514-batch/tmp')
)

p1.run()

/Users/prabha/opt/anaconda3/envs/beam/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1692: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  is_streaming_pipeline = p.options.view_as(StandardOptions).streaming
/Users/prabha/opt/anaconda3/envs/beam/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
